<a href="https://colab.research.google.com/github/mrdbourke/pytorch-deep-learning/blob/main/extras/exercises/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:** 

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [3]:
%%writefile going_modular/get_data.py
"""
contains data setup functionality
"""
from torchvision import datasets 
from torch.utils.data import DataLoader
from pathlib import Path 
from zipfile import ZipFile
import requests

DATA_PATH = Path("going_modular/data")
DATA_PATH.mkdir(parents=True, exist_ok=True)

IMAGE_FILE = "pizza_sushi_steak.zip"
IMAGE_DIR = DATA_PATH / IMAGE_FILE
if IMAGE_DIR.is_file():
    print("image datasets already exist")
else:
    print("creating dataset directory..")
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
    with open(DATA_PATH / IMAGE_FILE, "wb") as f: 
        f.write(request.content)

FILE_PATH = DATA_PATH / "pizza_sushi_steak"
if FILE_PATH.is_dir():
    print("file already extracted")
else:
    print("extracting..")
    with ZipFile(IMAGE_DIR, "r") as zip_ref:
        zip_ref.extractall(FILE_PATH)

Writing going_modular/get_data.py


In [6]:
# Example running of get_data.py
!python going_modular/get_data.py

image datasets already exist
file already extracted


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular). 

In [27]:
%%writefile going_modular/train.py
import os 
import torch 
import data_setup, engine, model_builder, utils
from torchvision import transforms 
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("-e", "--num_epochs", help="an integer to perform number of epochs", type=int)
parser.add_argument("-b", "--batch_size", help="an integer of number of element per batch", type=int)
parser.add_argument("-hu", "--hidden_units", help="an integer of number of hidden units per layer", type=int)
parser.add_argument("-lr", "--learning_rate", help="a float for the learning rate", type=float)

args = parser.parse_args()

# setup hyperparameters 
NUM_EPOCHS = args.num_epochs if args.num_epochs else 10
BATCH_SIZE = args.batch_size # required
HIDDEN_UNITS = args.hidden_units if args.hidden_units else 10
LEARNING_RATE = args.learning_rate if args.learning_rate else 0.001

# setup directories 
train_dir = "data/pizza_sushi_steak/train"
test_dir = "data/pizza_sushi_steak/test"

def main():
    # setup device agnostic code 
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # create transforms
    data_transform = transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.ToTensor()
    ])

    # create DataLoaders with help from data_setup.py
    train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
        train_dir=train_dir, 
        test_dir=test_dir, 
        transform=data_transform, 
        batch_size=BATCH_SIZE
    )

    # create model with help from model_builder.py
    model = model_builder.TinyVGG(
        input_shape=3, 
        hidden_units=HIDDEN_UNITS, 
        output_shape=len(class_names)
    ).to(device)

    # set loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

    # start training with help from engine.py 
    engine.train(model=model, 
                    train_dataloader=train_dataloader, 
                    test_dataloader=test_dataloader, 
                    loss_fn=loss_fn, 
                    optimizer=optimizer, 
                    epochs=NUM_EPOCHS, 
                    device=device)
            
    # save the model with help from utils.py
    utils.save_model(model=model, target_dir="models", model_name="05_going_modular_script_mode_tinyvgg_model_exc.pt")

if __name__ == '__main__':
    main()

Overwriting going_modular/train.py


In [8]:
# Example running of train.py
!python going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

Epoch: 1 | train_loss: 1.0995 | train_acc: 0.3673 | test_loss: 1.0945 | test_acc: 0.1875
[INFO] Saving model to: models\05_going_modular_script_mode_tinyvgg_model_exc.pt



100%|██████████| 5/5 [07:58<00:00, 95.80s/it]


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function). 
* You may also have to write code to load in a trained model.

In [27]:
%%writefile going_modular/predict.py
import argparse
import torch 
import matplotlib.pyplot as plt 
import requests
from PIL import Image
from torchvision import transforms
import data_setup, model_builder

parser = argparse.ArgumentParser()
parser.add_argument("-i", "--image", help="string of url to the image", type=str)
args = parser.parse_args()

URL = args.image # required

tf = transforms.Compose([
        transforms.Resize(size=(64, 64)),
        transforms.ToTensor()
])

# setup directories 
train_dir = "going_modular/data/pizza_sushi_steak/train"
test_dir = "going_modular/data/pizza_sushi_steak/test"

_, _, class_names = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir, transform=tf, batch_size=32)

# load saved model 
loaded_model = model_builder.TinyVGG(input_shape=3, hidden_units=128, output_shape=len(class_names))
loaded_model.load_state_dict(torch.load("models/05_going_modular_script_mode_tinyvgg_model_exc.pt", weights_only=True))

def pred_and_plot(model: torch.nn.Module, 
                    image_path: str,
                    transform: transforms.Compose,
                    class_names: list[str] = None):
        # load image
        img = Image.open(requests.get(image_path, stream=True).raw).convert("RGB")
        # setup transformed image 
        transformed_img = transform(img)
        # forward pass 
        logits = model(transformed_img.unsqueeze(dim=0))
        pred = torch.softmax(logits, dim=-1).argmax(dim=-1)
        # plot the image along with the label 
        plt.imshow(transformed_img.permute(1, 2, 0))
        title = f"{class_names[pred]} | {torch.softmax(logits, dim=-1).squeeze()[pred].item():.3f}"
        plt.title(title)
        print(title)

pred_and_plot(model=loaded_model, image_path=URL, 
                transform=tf, class_names=class_names)

Overwriting going_modular/predict.py


In [31]:
# Example running of predict.py 
!python going_modular/predict.py --image https://asset.kompas.com/crops/ygZlQa2c9c9GcnF3CDDfQuU4vFE=/0x0:1000x667/750x500/data/photo/2022/12/27/63aa85776cf9e.jpg

sushi | 0.426
